In [1]:
import numpy as np
from utils.data import build_dataloader_from_yaml
from torch.utils.data import Dataset, DataLoader
import torch
import csv

import torch.nn.functional as F
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
loader = build_dataloader_from_yaml("/home/xcg/medical-research/Project23us/config/train.yaml")

In [3]:
cnt = 0
for clip_feature, sam_feature, caption in loader:
    cnt += 1
    if cnt == 3:
        break

pairlen:  6
index:  3
index:  4
index:  5
index:  1
pairlen:  5
index:  2
index:  1
index:  3
index:  0
pairlen:  7
index:  6
index:  5
index:  0
index:  1


In [51]:

dataset_base = "/data2/xcg_data/lavis_data/2023us/features"
csv_path = "/data/xcg/lavis_data/coco-2023us/excels/translated.csv"

class Dataset_cls(Dataset):

    def __init__(self, dataset_base, csvpath, limitation = 4):
        self.csvpath = csvpath
        self.dataset_base = dataset_base
        self.pairs, self.keylist = self.load_caption()
        self.limitation = limitation
    
    def load_caption(self):

        # {personid: [[image_ids, ], caption], }
        pairs = {}
        with open(self.csvpath, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                personid = str(row[0]).split("_")[1]
                if personid not in pairs:
                    pairs[personid] = [[row[0]], row[1]]
                else:
                    pairs[personid][0].append(row[0])

        return pairs, list(pairs.keys())


    def __len__(self):
        return len(self.keylist)

    def __getitem__(self, index):
        # clip_feature, sam_feature, caption
        personid = self.keylist[index]
        pairlen = len(self.pairs[personid][0])
        cls_list = []
        imgid_list = []
        cls_dict = {}
        for i in range(pairlen):
            clip_feature_path = self.dataset_base + "/clip_features/" + self.pairs[personid][0][i] + '.npz'
            clip_dataloads = np.load(clip_feature_path)
            clip_cls = torch.from_numpy(clip_dataloads["arr"])[0]
            cls_list.append(clip_cls)
            imgid_list.append(self.pairs[personid][0][i])
            # cls_dict[self.pairs[personid][0][i]] = clip_cls

        return cls_list, imgid_list
        # return cls_dict

In [52]:
myset = Dataset_cls( dataset_base, csv_path, limitation = 4)

In [6]:
myset[0][0][0].shape

torch.Size([1408])

In [54]:
for clip_cls, imgid in myset:
    print(clip_cls)
    print(imgid)
    break

[tensor([ 0.0717,  0.0560, -0.4620,  ...,  0.1498,  0.6572, -0.0380]), tensor([-0.0921,  0.0730, -0.4331,  ...,  0.0272,  0.6542,  0.0873]), tensor([ 0.0191,  0.0920, -0.4268,  ...,  0.0809,  0.6470, -0.1222]), tensor([-0.0272,  0.0735, -0.3529,  ...,  0.1641,  0.6196, -0.2456]), tensor([ 0.1014,  0.1429, -0.4235,  ...,  0.1438,  0.6310, -0.1794]), tensor([ 0.1037,  0.0642, -0.4444,  ...,  0.0939,  0.6034,  0.1693])]
['20230212_11702789_154720983', '20230212_11702789_154745899', '20230212_11702789_154910734', '20230212_11702789_154707151', '20230212_11702789_154713016', '20230212_11702789_154750553']


In [59]:
for clip_cls, imgid in myset:
    if len(clip_cls) < 8:
        continue
    
    vectors = torch.stack(clip_cls, dim=0)
    
    normalized_vectors = F.normalize(vectors, p=2, dim=1)
    
    for i in range(normalized_vectors.shape[0]):
        print(normalized_vectors[i])

    normalized_vectors = normalized_vectors.numpy()

    kmeans = KMeans(n_clusters=8, random_state=0)

    cluster_labels = kmeans.fit_predict(normalized_vectors)

    cluster_centers = kmeans.cluster_centers_
    
    cosine_sims = cosine_similarity(normalized_vectors, cluster_centers)

    for i in range(8):
        cluster_indices = np.where(cluster_labels == i)[0]
        cluster_similarities = cosine_sims[cluster_indices, i]
        representative_index = cluster_indices[np.argmax(cluster_similarities)]
        # print(imgid[representative_index])
        print(representative_index)

    break


tensor([ 0.0036,  0.0097, -0.0148,  ...,  0.0228,  0.0361,  0.0104])
tensor([ 0.0059,  0.0135, -0.0190,  ...,  0.0308,  0.0457,  0.0092])
tensor([ 0.0031,  0.0065, -0.0253,  ...,  0.0315,  0.0400,  0.0053])
tensor([ 0.0033,  0.0040, -0.0228,  ...,  0.0277,  0.0474,  0.0047])
tensor([ 0.0056,  0.0023, -0.0177,  ...,  0.0285,  0.0393,  0.0032])
tensor([ 0.0059,  0.0103, -0.0221,  ...,  0.0304,  0.0406,  0.0034])
tensor([ 0.0059,  0.0044, -0.0338,  ...,  0.0325,  0.0456,  0.0045])
tensor([-0.0025,  0.0059, -0.0350,  ...,  0.0356,  0.0409,  0.0063])
tensor([ 0.0037,  0.0003, -0.0333,  ...,  0.0308,  0.0429, -0.0003])
tensor([ 0.0145, -0.0008, -0.0216,  ...,  0.0221,  0.0250,  0.0035])
tensor([ 0.0023,  0.0123, -0.0217,  ...,  0.0352,  0.0428,  0.0054])
4
8
7
10
6
3
9
0


/home/xcg/anaconda3/envs/medical/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [1]:
import json
with open("/home/xcg/medical-research/Project23us/labels/8000patient.json", 'r') as f:
    data = json.load(f)
    

In [4]:
import numpy as np
a = np.ones(11)

In [18]:
import torch.nn as nn
import torch
alist = nn.ModuleList([nn.Linear(6, 3) for _ in range(6)])

In [19]:
inputs = torch.rand(6)

In [20]:
blist = []
for i in range(4):
    blist.append(alist[i](inputs))
x = torch.stack(blist)

In [21]:
x.shape

torch.Size([4, 3])